# Roteamento de cidades com Busca A*

Na busca A* é usado a heurística h(n) (custo até o objetivo) em conjunto com a heurística grafo(n) (custo de cada nó) formando:

*f(n) = h(n) + grafo(n)*

### Modelando Nó e Grafo:

In [1]:
class Grafo:
  def __init__(self, dict_grafo=None, direcionado=True):
    self.dict_grafo = dict_grafo or {}
    self.direcionado = direcionado
    if not direcionado:
      self.tornar_nao_direcionado()

  def tornar_nao_direcionado(self):
    for a in list(self.dict_grafo.keys()):
      for (b, dist) in self.dict_grafo[a].items():
        self.dict_grafo.setdefault(b, {})[a] = dist

  def conectar(self, A, B, distancia=1):
    self.dict_grafo.setdefault(A, {})[B] = distancia
    if not self.direcionado:
      self.dict_grafo.setdefault(B, {})[A] = distancia

  def get(self, a, b=None):
    links = self.dict_grafo.setdefault(a, {})
    if b is None:
      return links
    else:
      return links.get(b)

  def nos(self):
    s1 = set([k for k in self.dict_grafo.keys()])
    s2 = set([k2 for v in self.dict_grafo.values() for k2, v2 in v.items()])
    nos = s1.union(s2)
    return list(nos)

class No:
  def __init__(self, name:str, parent:str):
    self.name = name
    self.parent = parent
    self.g = 0 # distancia para o nó inicial
    self.h = 0 # distancia para o nó final

  def get_f(self):
    return self.g + self.h

  def __eq__(self, other):
    return self.name == other.name
  
  def __lt__(self, other):
    return self.f < other.f
  
  def __repr__(self):
    return ('({0},{1})'.format(self.name, self.get_f()))

  def __hash__(self):
    return hash(self.name)

### Implementando a Busca A*:

In [2]:
import heapq

def obter_caminho(no_atual, inicial):
  '''
    Função para gerar o caminho percorrido entre dois estados
    Args:
        - no_atual: estado no_atual
        - inicial: estado inicial
    Return:
        - Caminho percorrido entre o estado inicial e final
  '''
  caminho = []
  while no_atual != inicial:
    caminho.append(no_atual.name + ': ' + str(no_atual.g))
    no_atual = no_atual.parent
  caminho.append(inicial.name + ': ' + str(inicial.g))

  return caminho[::-1]

def a_estrela(grafo: Grafo, heuristicas, inicio, fim):
  '''
    Algoritmo a*, dado um grafo, um cojunto de heurísticas, um estado/nó inicial e um estado/nó final, 
    busca o caminho ótimo e completo
    Args:
        - grafo: grafo contendo os estados da busca
        - heuristicas: heuristica de cada estado
        - inicio: estado inicial
        - fim: estado final
    Return:
        - Melhor caminho entre estados inicial e final, se a solução foi encontrada, quantidade de estados visitados
  '''
    
  # Criando nó inicial e nó objetivo (não possuem pai)
  no_inicial = No(inicio, None)
  no_objetivo = No(fim, None)
  
  # Inicializando os atributos grafo(0) e h(0)
  no_inicial.grafo = 0
  no_inicial.h = heuristicas[inicio]

  id = 0

  # Fila de prioridades
  fila = []
  heapq.heappush(fila, (no_inicial.get_f(), id, no_inicial))

  # Nós criados
  nos_criados = []
  nos_criados.append(no_inicial)

  # Nós visitados
  nos_visitados = []
  
  # Controle
  solucao_encontrada = False
  caminho = None

  # Loop enquanto a fila não estiver vazia.
  while len(fila) != 0:
    # Pega na fila proxímo nó.
    no_atual = heapq.heappop(fila)[2]
    nos_visitados.append(no_atual)
    
    # Checar se já chegou na solução.
    if no_atual == no_objetivo:
      caminho = obter_caminho(no_atual, no_inicial)
      solucao_encontrada = True
      break

    # Percorrendo todos os vizinhos do nó atual.
    for nome, g in grafo.get(no_atual.name).items():
      # Criando nó com o nome do vizinho e passando o nó atual como pai.
      no_vizinho = No(nome, no_atual)

      # Checar se o nó vizinho já não foi criado.
      if no_vizinho not in nos_criados:
        # Atribuir valor de h(n) e g(n) ao nó.
        no_vizinho.g = no_atual.g + g
        no_vizinho.h = heuristicas[no_vizinho.name]
        
        # Inserir nó na lista de criados.
        nos_criados.append(no_vizinho)

        # Inserir nó na fila.
        id += 1
        heapq.heappush(fila, (no_vizinho.get_f(), id, no_vizinho))
      
  return solucao_encontrada, caminho, nos_visitados 

### Modelando e executando problema:

![2](imagens/2.png)

In [3]:
grafo = Grafo()

grafo.conectar('Santa Rita do Sapucaí', 'Pouso Alegre', 28.9)
grafo.conectar('Pouso Alegre', 'Cambui', 49.1)
grafo.conectar('Pouso Alegre', 'Congonhal', 24.3)
grafo.conectar('Pouso Alegre', 'Borda', 28.8)
grafo.conectar('Cambui', 'Camanducaia', 24.7)
grafo.conectar('Camanducaia', 'Braganca', 60.4)
grafo.conectar('Braganca', 'Atibaia', 25.2)
grafo.conectar('Braganca', 'Itapira', 82.4)
grafo.conectar('Atibaia', 'Campinas', 65.6)
grafo.conectar('Itapira', 'Campinas', 70.7)
grafo.conectar('Borda', 'Jacutinga', 57.6)
grafo.conectar('Jacutinga', 'Itapira', 33.2)
grafo.conectar('Congonhal', 'Ipuiuna', 24.6)
grafo.conectar('Ipuiuna', 'Andradas', 67.5)
grafo.conectar('Andradas', 'Espírito Santo do Pinhal', 28.4)
grafo.conectar('Espírito Santo do Pinhal', 'Mogi Guaçu', 35.7)
grafo.conectar('Mogi Guaçu', 'Mogi Mirim', 25)
grafo.conectar('Mogi Mirim', 'Campinas', 60.1)

grafo.tornar_nao_direcionado()

h = {}
h['Santa Rita do Sapucaí'] = 165
h['Pouso Alegre'] = 137
h['Cambui'] = 108
h['Camanducaia'] = 97
h['Braganca'] = 54
h['Atibaia'] = 57
h['Campinas'] = 0
h['Borda'] = 117
h['Jacutinga'] = 84
h['Itapira'] = 58
h['Congonhal'] = 135
h['Ipuiuna'] = 139
h['Andradas'] = 106
h['Espírito Santo do Pinhal'] = 86
h['Mogi Guaçu'] = 62
h['Mogi Mirim'] = 54

solucao_encontrada, caminho, nos_visitados = a_estrela(grafo, h, 'Santa Rita do Sapucaí', 'Campinas')

from pprint import pprint

print(f'Solução encontrada? {solucao_encontrada}')
print('Caminho encontrado:') 
pprint(caminho)
print('Cidades (nós) visitados:')
pprint(nos_visitados)

Solução encontrada? True
Caminho encontrado:
['Santa Rita do Sapucaí: 0',
 'Pouso Alegre: 28.9',
 'Borda: 57.7',
 'Jacutinga: 115.30000000000001',
 'Itapira: 148.5',
 'Campinas: 219.2']
Cidades (nós) visitados:
[(Santa Rita do Sapucaí,165),
 (Pouso Alegre,165.9),
 (Borda,174.7),
 (Cambui,186.0),
 (Congonhal,188.2),
 (Jacutinga,199.3),
 (Camanducaia,199.7),
 (Itapira,206.5),
 (Ipuiuna,216.8),
 (Braganca,217.1),
 (Campinas,219.2)]


![3](imagens/3.png)